In [ ]:
# Citibike daily trip count by station

from datetime import date, datetime, timedelta
import pandas as pd
import csv
import os

In [ ]:
rootdir = 'C:/Users/geoff/PycharmProjects/citibike/citibike_trip_data/citibike_all_months/'

In [ ]:
for file in os.listdir(rootdir):
    if file.endswith(".csv"):
        df = pd.read_csv(rootdir + file,
                         header=0,
                         names=['trip_duration', 'trip_start_time', 'trip_end_time',
                                'start_station_id', 'start_station_name', 'start_station_lat', 'start_station_lon',
                                'end_station_id', 'end_station_name', 'end_station_lat', 'end_station_lon',
                                'bike_id', 'user_type', 'birth_year', 'gender'
                                ],
                         dtype={'trip_duration': 'Int64', 'trip_start_time': 'object', 'trip_end_time': 'object',
                                'start_station_id': 'Int64', 'start_station_name': 'object',
                                'start_station_lat': 'float64', 'start_station_lon': 'float64',
                                'end_station_id': 'Int64', 'end_station_name': 'object',
                                'end_station_lat': 'float64', 'end_station_lon': 'float64',
                                'bike_id': 'Int64', 'user_type': 'object', 'birth_year': 'Int64', 'gender': 'Int64'},
                         engine='python'
                         )
        df = df.dropna(subset=['start_station_id', 'start_station_name', 'end_station_id', 'end_station_name'])
        trip_list = df.values.tolist()

        trip_dict = {}

        for line in trip_list:
            date = datetime.strptime(line[1], '%Y-%m-%d %H:%M:%S.%f' ).date()
            start_station_id = line[3]
            start_station_name = str(line[4])
            start_station_lat = line[5]
            start_station_lon = line[6]
            end_station_id = line[7]
            end_station_name = str(line[8])
            end_station_lat = line[9]
            end_station_lon = line[10]

            # count trip starts and ends
            # collate trip counts in dicts
            if date not in trip_dict:
                trip_dict[date] = {}
            if start_station_id not in trip_dict[date]:
                trip_dict[date][start_station_id] = [1, 0,
                                                     start_station_name,
                                                     start_station_lat,
                                                     start_station_lon
                                                     ]
            else:
                trip_dict[date][start_station_id][0] += 1
            if end_station_id not in trip_dict[date]:
                trip_dict[date][end_station_id] = [0, 1,
                                                   end_station_name,
                                                   end_station_lat,
                                                   end_station_lon
                                                   ]
            else:
                trip_dict[date][end_station_id][1] += 1

        # create a list of trip counts by date and station
        trip_list_summary = []
        trip_list_formatted = []

        for trip_date in trip_dict.keys():
            for station_id in trip_dict[trip_date].keys():
                trip_list_summary = [trip_date, station_id,
                                     trip_dict[trip_date][station_id][0], trip_dict[trip_date][station_id][1],
                                     trip_dict[trip_date][station_id][2], trip_dict[trip_date][station_id][3],
                                     trip_dict[trip_date][station_id][4]]
                trip_list_formatted.append(trip_list_summary)

        # sort the file
        trip_list_formatted.sort(key=lambda i: (i[0], int(i[1])))

        # create csv file
        file_ext = file[:6]
        columns = ["date", "station_id", "trip_starts", "trip_ends", "station_name", "station_lat", "station_lon"]
        df = pd.DataFrame(trip_list_formatted, columns=columns)

        # create csv in test folder
        df.to_csv("C:/Users/geoff/PycharmProjects/citibike/citibike_trip_data/citibike_summary_trip_counts/citibike_station_trip_counter_{}.csv".format(file_ext),
                  quoting=csv.QUOTE_NONE,
                  index=False)